### Compare MRNs in PET and ECHO reports ###
This notebook just compares PET studies and ECHO videos on the file system. 

In [1]:
import os
import pandas as pd

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)

In [2]:
cfr_data_root = os.path.normpath('/mnt/obi0/andreas/data/cfr')
cfr_meta_date = '200202'
meta_dir = os.path.join(cfr_data_root, 'metadata_'+cfr_meta_date)
print(meta_dir)

/mnt/obi0/andreas/data/cfr/metadata_200202


#### PET REPORTS ####

In [3]:
# PET REPORTS
pet_reports_file = os.path.join(cfr_data_root, '210_getStressTest.xlsx')
pet_df = pd.read_excel(pet_reports_file)
keep_cols = ['reportID', 'mrn', 'subjectid', 'report_number', 'study_date', 'report_date', 'cfr']
pet = pet_df[keep_cols]
pet.loc[pet.study_date.index].study_date = pd.to_datetime(pet.study_date)
pet.loc[pet.report_date.index].study_date = pd.to_datetime(pet.report_date)

print('Number of reports: {}'.format(len(pet.report_number.unique())))
print('Unique patients:   {}'.format(len(pet.mrn.unique())))
print('Unique cfr values: {}'.format(len(pet.cfr.unique())))

# Lets keep only a few columns 
pet.head()

# Additional PET studies MRNs
pet_studies_needed = os.path.join(cfr_data_root, 'pet_studies_needed.txt')
pet_df_needed = pd.read_csv(pet_studies_needed, sep = '\t', header = None)
pet_df_needed.columns = ['mrn', 'study_date']
pet_df_2 = pet_df.merge(right = pet_df_needed, on = ['mrn', 'study_date'], how = 'outer')

Number of reports: 3350
Unique patients:   3072
Unique cfr values: 3414


In [4]:
pet_df_3 = pet_df_2[['mrn', 'study_date', 'report_number']].groupby('mrn').first().\
            reset_index(drop = False)

In [5]:
print(len(pet_df_2.mrn.unique()))
print(len(pet_df_3.mrn.unique()))

3755
3755


In [6]:
pet_df_3.head(5)

,mrn,study_date,report_number
0,9241,28-Jan-10,EVS0228610
1,59691,02-Oct-13,EVS0419689
2,68189,22-Oct-13,EVS0446758
3,81182,17-Aug-07,0721092P
4,88435,20-Feb-15,E1789200


#### NPY ECHO DATA ####

In [7]:
echo_files_name = 'echo_npyFiles_BWH_'+cfr_meta_date+'.parquet'
echo = pd.read_parquet(os.path.join(meta_dir, echo_files_name))
echo = echo.astype({'mrn': 'int64'})
echo = echo.assign(echo_study_date = echo.datetime.dt.date,
                   echo_study_time = echo.datetime.dt.time)

In [8]:
print('Number of npy files: {}'.format(len(echo.filename.unique())))
print('Unique echo studies: {}'.format(len(echo.study.unique())))
print('Unique patients:     {}'.format(len(echo.mrn.unique())))
print()
echo.head()

Number of npy files: 1029179
Unique echo studies: 20525
Unique patients:     12000



,filename,dir,study,mrn,datetime,echo_study_date,echo_study_time
0,4a1ed563ca4f684e_4903a580509f95be6ede48ce4fec_...,/mnt/obi0/phi/echo/npyFiles/BWH/4a1e/4a1ed563c...,4a1ed563ca4f684e_4903a580509f95be6ede48ce4fec,18789404,2015-08-11 16:05:24,2015-08-11,16:05:24
1,4a1ed563ca4f684e_4903a580509f95be6ede48ce4fec_...,/mnt/obi0/phi/echo/npyFiles/BWH/4a1e/4a1ed563c...,4a1ed563ca4f684e_4903a580509f95be6ede48ce4fec,18789404,2015-08-11 16:05:24,2015-08-11,16:05:24
2,4a1ed563ca4f684e_4903a580509f95be6ede48ce4fec_...,/mnt/obi0/phi/echo/npyFiles/BWH/4a1e/4a1ed563c...,4a1ed563ca4f684e_4903a580509f95be6ede48ce4fec,18789404,2015-08-11 16:05:24,2015-08-11,16:05:24
3,4a1ed563ca4f684e_4903a580509f95be6ede48ce4fec_...,/mnt/obi0/phi/echo/npyFiles/BWH/4a1e/4a1ed563c...,4a1ed563ca4f684e_4903a580509f95be6ede48ce4fec,18789404,2015-08-11 16:05:24,2015-08-11,16:05:24
4,4a1ed563ca4f684e_4903a580509f95be6ede48ce4fec_...,/mnt/obi0/phi/echo/npyFiles/BWH/4a1e/4a1ed563c...,4a1ed563ca4f684e_4903a580509f95be6ede48ce4fec,18789404,2015-08-11 16:05:24,2015-08-11,16:05:24


#### PET STUDIES + NPY ECHO DATA ####

In [9]:
pet_echo = pet_df_3.merge(right = echo, on = 'mrn', how = 'left')
pet_echo.tail()

,mrn,study_date,report_number,filename,dir,study,datetime,echo_study_date,echo_study_time
200290,40604613,2019-11-20,NaN,NaN,NaN,NaN,NaT,NaN,NaN
200291,40611006,2019-10-17,NaN,NaN,NaN,NaN,NaT,NaN,NaN
200292,40768871,2019-12-11,NaN,NaN,NaN,NaN,NaT,NaN,NaN
200293,40902504,2019-11-06,NaN,NaN,NaN,NaN,NaT,NaN,NaN
200294,41128695,2020-01-10,NaN,NaN,NaN,NaN,NaT,NaN,NaN


In [10]:
# Filter pet studies with current echo
pet_echo_nofiles = pet_echo.loc[pet_echo.filename.isnull()].reset_index(drop = True)
pet_echo_files = pet_echo.loc[~pet_echo.filename.isnull()].reset_index(drop = True)

In [11]:
pet_echo_nofiles.tail()

,mrn,study_date,report_number,filename,dir,study,datetime,echo_study_date,echo_study_time
1683,40604613,2019-11-20,NaN,NaN,NaN,NaN,NaT,NaN,NaN
1684,40611006,2019-10-17,NaN,NaN,NaN,NaN,NaT,NaN,NaN
1685,40768871,2019-12-11,NaN,NaN,NaN,NaN,NaT,NaN,NaN
1686,40902504,2019-11-06,NaN,NaN,NaN,NaN,NaT,NaN,NaN
1687,41128695,2020-01-10,NaN,NaN,NaN,NaN,NaT,NaN,NaN


In [12]:
print('PET with ECHO:')
print('Number of npy files: {}'.format(len(pet_echo_files.filename.unique())))
print('Unique echo studies: {}'.format(len(pet_echo_files.study.unique())))
print('Unique patients:     {}'.format(len(pet_echo_files.mrn.unique())))

PET with ECHO:
Number of npy files: 198607
Unique echo studies: 4274
Unique patients:     2067


In [13]:
print('PET without ECHO:')
print('Unique patients:     {}'.format(len(pet_echo_nofiles.mrn.unique())))
print('Total PET patients:  {}'.format(len(pet_df_3.mrn.unique())))

PET without ECHO:
Unique patients:     1688
Total PET patients:  3755


In [73]:
pet_echo_nofiles.head()

,mrn,study_date,report_number,filename,dir,study,datetime,echo_study_date,echo_study_time
0,68189,22-Oct-13,EVS0446758,NaN,NaN,NaN,NaT,NaN,NaN
1,81182,17-Aug-07,0721092P,NaN,NaN,NaN,NaT,NaN,NaN
2,137091,28-Apr-09,EVS0182113,NaN,NaN,NaN,NaT,NaN,NaN
3,142232,12-Sep-08,0827928A,NaN,NaN,NaN,NaT,NaN,NaN
4,178046,2019-01-10,NaN,NaN,NaN,NaN,NaT,NaN,NaN


#### ECHO STUDIES ON DCM BUT NOT CONVERTED TO NPY ####

In [15]:
dcm_studies_file

'echo_deIdentifyedEcho_BWH_dcm_200202.parquet'

In [14]:
dcm_studies_file = 'echo_deIdentifyedEcho_BWH_dcm_'+cfr_meta_date+'.parquet'
dcm = pd.read_parquet(os.path.join(meta_dir, dcm_studies_file))
dcm = dcm.astype({'mrn': 'int64'})

dcm.head()

,mrn,reportID,subjectid,report_number,study_date,report_date,cfr,filename,dir,study,datetime
0,142232,90848,434,0827928A,12-Sep-08,12-Sep-08,1.443478,Image-16.dcm,/mnt/obi0/phi/echo/deIdentifyedEcho/BWH/4b7b/4...,4b7b475df4b17b04_4903a44b32e5a539f80e6819b98b,2008-09-12 14:45:49
1,288407,157776,7446,E6330362,12-Oct-17,07-Oct-17,1.844106,Image-9.dcm,/mnt/obi0/phi/echo/deIdentifyedEcho/BWH/4b7b/4...,4b7b44d7d346b8b3_4903a582edf18fdf650ef98a8e6d,2017-10-07 10:30:38
2,794941,72359,2605,EVS0182499,11-Jul-08,28-Apr-09,1.615031,Image-37.dcm,/mnt/obi0/phi/echo/deIdentifyedEcho/BWH/4b7b/4...,4b7b41a5f60a4ebe_4903a44ab12fcc860f4f20cd0ef8,2009-04-28 13:05:07
3,813071,85015,452,0806735A,09-Apr-08,05-Mar-08,2.198669,Image-16.dcm,/mnt/obi0/phi/echo/deIdentifyedEcho/BWH/4b7b/4...,4b7b4edb2e8d49d5_4903a44b32efc00c83a4de086041,2008-03-05 13:13:20
4,1145762,102950,180,EVS0245099,03-May-10,04-May-10,1.879962,Image-2.dcm,/mnt/obi0/phi/echo/deIdentifyedEcho/BWH/4b7a/4...,4b7a8652c32ed64d_4903a585842b96fd35619b87b41a,2010-05-04 08:40:03


In [79]:
rm_cols = list(echo.columns)
rm_cols.remove('mrn')
pet_echo_nofiles_x = pet_echo_nofiles.drop(columns = rm_cols)

# These are CFR studies without .npy files
pet_echo_nofiles_x.head()

,mrn,study_date,report_number
0,68189,22-Oct-13,EVS0446758
1,81182,17-Aug-07,0721092P
2,137091,28-Apr-09,EVS0182113
3,142232,12-Sep-08,0827928A
4,178046,2019-01-10,NaN


In [76]:
# Merge CFR studies (without .npy) with _dcm data
pet_echo_nofiles_dcm = pet_echo_nofiles_x.merge(right = dcm, on = 'mrn', how = 'left')
pet_echo_nofiles_dcm.head()

,mrn,study_date_x,report_number_x,reportID,subjectid,report_number_y,study_date_y,report_date,cfr,filename,dir,study,datetime
0,68189,22-Oct-13,EVS0446758,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,81182,17-Aug-07,0721092P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2,137091,28-Apr-09,EVS0182113,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3,142232,12-Sep-08,0827928A,90848.0,434.0,0827928A,12-Sep-08,12-Sep-08,1.443478,Image-16.dcm,/mnt/obi0/phi/echo/deIdentifyedEcho/BWH/4b7b/4...,4b7b475df4b17b04_4903a44b32e5a539f80e6819b98b,2008-09-12 14:45:49
4,178046,2019-01-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [77]:
# We have many files for each mrn. 
# But we only want to know the mrn numbers for which have dcm data.
pet_echo_nofiles_dcm = pet_echo_nofiles_dcm.groupby('mrn').first().reset_index(drop = False)
pet_echo_nofiles_dcm.head()

,mrn,study_date_x,report_number_x,reportID,subjectid,report_number_y,study_date_y,report_date,cfr,filename,dir,study,datetime
0,68189,22-Oct-13,EVS0446758,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,81182,17-Aug-07,0721092P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2,137091,28-Apr-09,EVS0182113,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3,142232,12-Sep-08,0827928A,90848.0,434.0,0827928A,12-Sep-08,12-Sep-08,1.443478,Image-16.dcm,/mnt/obi0/phi/echo/deIdentifyedEcho/BWH/4b7b/4...,4b7b475df4b17b04_4903a44b32e5a539f80e6819b98b,2008-09-12 14:45:49
4,178046,2019-01-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [21]:
#pet_echo_nofiles = pet_echo.loc[pet_echo.filename.isnull()].reset_index(drop = True)
df_pet_echo_nodcm = pet_echo_nofiles_dcm.loc[pet_echo_nofiles_dcm.filename.isnull()].reset_index(drop=True)
df_pet_echo_dcm = pet_echo_nofiles_dcm.loc[~pet_echo_nofiles_dcm.filename.isnull()].reset_index(drop=True)

# Save those
df_pet_echo_nodcm_file = 'pet_echo_deIdentifyedEcho_BWH_nodcm_'+cfr_meta_date+'.parquet'
df_pet_echo_dcm_file = 'pet_echo_deIdentifyedEcho_BWH_dcm_'+cfr_meta_date+'.parquet'
df_pet_echo_nodcm.to_parquet(os.path.join(meta_dir, df_pet_echo_nodcm_file))
df_pet_echo_dcm.to_parquet(os.path.join(meta_dir, df_pet_echo_dcm_file))

In [22]:
df_pet_echo_nodcm.head()

,mrn,reportID,subjectid,report_number,study_date,report_date,cfr,filename,dir,study,datetime
0,68189,127394,5580,EVS0446758,22-Oct-13,22-Oct-13,1.504065,NaN,NaN,NaN,NaT
1,81182,86436,3149,0721092P,17-Aug-07,13-Aug-07,1.720486,NaN,NaN,NaN,NaT
2,137091,94942,1777,EVS0182113,28-Apr-09,27-Apr-09,0.790368,NaN,NaN,NaN,NaT
3,240762,86216,751,0629-0010,14-May-07,29-Jun-06,1.405966,NaN,NaN,NaN,NaT
4,337014,136733,6194,EVS0545366,13-Jan-15,13-Jan-15,1.115566,NaN,NaN,NaN,NaT


In [23]:
print('MRNs that are not available as .dcm files: {}'.format(len(df_pet_echo_nodcm.mrn.unique())))

MRNs that are not available as .dcm files: 940


In [24]:
df_pet_echo_dcm.head()

,mrn,reportID,subjectid,report_number,study_date,report_date,cfr,filename,dir,study,datetime
0,142232,90848,434,0827928A,12-Sep-08,12-Sep-08,1.443478,Image-16.dcm,/mnt/obi0/phi/echo/deIdentifyedEcho/BWH/4b7b/4...,4b7b475df4b17b04_4903a44b32e5a539f80e6819b98b,2008-09-12 14:45:49
1,288407,157776,7446,E6330362,12-Oct-17,07-Oct-17,1.844106,Image-9.dcm,/mnt/obi0/phi/echo/deIdentifyedEcho/BWH/4b7b/4...,4b7b44d7d346b8b3_4903a582edf18fdf650ef98a8e6d,2017-10-07 10:30:38
2,794941,72359,2605,EVS0182499,11-Jul-08,28-Apr-09,1.615031,Image-37.dcm,/mnt/obi0/phi/echo/deIdentifyedEcho/BWH/4b7b/4...,4b7b41a5f60a4ebe_4903a44ab12fcc860f4f20cd0ef8,2009-04-28 13:05:07
3,813071,85015,452,0806735A,09-Apr-08,05-Mar-08,2.198669,Image-16.dcm,/mnt/obi0/phi/echo/deIdentifyedEcho/BWH/4b7b/4...,4b7b4edb2e8d49d5_4903a44b32efc00c83a4de086041,2008-03-05 13:13:20
4,1145762,102950,180,EVS0245099,03-May-10,04-May-10,1.879962,Image-2.dcm,/mnt/obi0/phi/echo/deIdentifyedEcho/BWH/4b7a/4...,4b7a8652c32ed64d_4903a585842b96fd35619b87b41a,2010-05-04 08:40:03


In [25]:
print('Studies that are yet available as .dcm files: {}'.format(len(df_pet_echo_dcm.study.unique())))
print('MRNs that are available as .dcm files: {}'.format(len(df_pet_echo_dcm.mrn.unique())))

Studies that are yet available as .dcm files: 219
MRNs that are available as .dcm files: 219
